In [6]:
import pathlib
import textwrap
import os

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
fetched_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=fetched_api_key)
model = genai.GenerativeModel("models/gemini-1.5-flash")

In [ ]:
import PIL.Image

img = PIL.Image.open('image.jpg')
img

In [11]:
response = model.generate_content(["Write a short, engaging blog post based on this picture. It should include a description of the meal in the photo and talk about my journey meal prepping.", img], stream=True)
response.resolve()

In [12]:
to_markdown(response.text)

> ## My Meal Prep Journey: From Chaos to Calm
> 
> It's no secret that meal prepping is a game-changer.  Gone are the days of frantic weekday evenings, searching for something edible and healthy in the depths of my fridge.  Now, I have these beautiful little containers filled with delicious, nutritious meals ready to go.
> 
> This week's creation? **Teriyaki chicken with brown rice, broccoli, red peppers, and carrots.**  It's bursting with flavor and color, and it's just the right amount of food to keep me satisfied until my next meal. 
> 
> My meal prep journey wasn't always this smooth sailing.  There were definitely some rough patches, like the time I accidentally made enough food for a small army and the time I discovered I'd left my lunch in the fridge... with the door open.  But through trial and error (and a few hilarious mishaps), I've finally found a system that works for me.  
> 
> Meal prepping isn't just about saving time and money, it's about taking control of my health and well-being.  It allows me to make healthy choices without compromising on flavor or convenience.  And who knows, maybe one day I'll even be able to resist the siren call of the takeout menu. 😉


In [15]:
chat = model.start_chat(history=[])
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
        cached_content=None
    ),
    history=[]
)

In [16]:
response = chat.send_message("In one sentence, explain how a computer works to a young child.")
to_markdown(response.text)

> A computer is like a super-smart toy that follows your instructions, uses numbers to remember things, and shows you the results on a screen. 


In [17]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?", stream=True)

for chunk in response:
    print(chunk.text)
    print("_"*80)

A
________________________________________________________________________________
 computer is essentially a complex machine that processes information using a binary code of 0
________________________________________________________________________________
s and 1s, following instructions from software programs to perform calculations, store
________________________________________________________________________________
 data, and communicate with other devices. 

________________________________________________________________________________


In [18]:
chat.history

[parts {
   text: "In one sentence, explain how a computer works to a young child."
 }
 role: "user",
 parts {
   text: "A computer is like a super-smart toy that follows your instructions, uses numbers to remember things, and shows you the results on a screen. \n"
 }
 role: "model",
 parts {
   text: "Okay, how about a more detailed explanation to a high schooler?"
 }
 role: "user",
 parts {
   text: "A computer is essentially a complex machine that processes information using a binary code of 0s and 1s, following instructions from software programs to perform calculations, store data, and communicate with other devices. \n"
 }
 role: "model"]

In [19]:
for message in chat.history:
    display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: In one sentence, explain how a computer works to a young child.

> **model**: A computer is like a super-smart toy that follows your instructions, uses numbers to remember things, and shows you the results on a screen. 


> **user**: Okay, how about a more detailed explanation to a high schooler?

> **model**: A computer is essentially a complex machine that processes information using a binary code of 0s and 1s, following instructions from software programs to perform calculations, store data, and communicate with other devices. 


In [21]:
model.count_tokens("What is the meaning of life?")
model.count_tokens(chat.history)

total_tokens: 106

In [22]:
result = genai.embed_content(
    model="models/embedding-001",
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string")

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED]')

[-0.003216741, -0.013358698, -0.017649598, -0.0091 ... TRIMMED]


In [23]:
result = genai.embed_content(
    model="models/embedding-001",
    content=[
        'What is the meaning of life?',
        'How much wood would a woodchuck chuck?',
        'How does the brain work?'],
    task_type="retrieval_document",
    title="Embedding of list of strings")

# A list of inputs > A list of vectors output
for v in result['embedding']:
  print(str(v)[:50], '... TRIMMED ...')

[0.0040260437, 0.004124458, -0.014209415, -0.00183 ... TRIMMED ...
[-0.004049845, -0.0075574904, -0.0073463684, -0.03 ... TRIMMED ...
[0.025310587, -0.0080734305, -0.029902633, 0.01160 ... TRIMMED ...


In [24]:
response.candidates[0].content

parts {
  text: "A computer is essentially a complex machine that processes information using a binary code of 0s and 1s, following instructions from software programs to perform calculations, store data, and communicate with other devices. \n"
}
role: "model"

In [27]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = response.candidates[0].content)

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... TRIMMED ...')

[-0.002067016, -0.056467608, 0.01130487, 0.0133229 ... TRIMMED ...


In [28]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = chat.history)

# 1 input > 1 vector output
for i,v in enumerate(result['embedding']):
    print(str(v)[:50], '... TRIMMED...')

[-0.014632266, -0.042202696, -0.015757175, 0.01548 ... TRIMMED...
[-0.024947928, -0.04683991, -0.0035777327, 0.01660 ... TRIMMED...
[-0.010055617, -0.07208932, -0.00011750793, -0.023 ... TRIMMED...
[-0.002067016, -0.056467608, 0.01130487, 0.0133229 ... TRIMMED...


In [34]:
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(
    genai.protos.Content(
        parts = [
            genai.protos.Part(text="Write a short, engaging blog post based on this picture."),
            genai.protos.Part(
                inline_data=genai.protos.Blob(
                    mime_type='image/jpeg',
                    data=pathlib.Path('image1.jpg').read_bytes()
                )
            ),
        ],
    ),
    stream=True)

In [35]:
response.resolve()

In [36]:
to_markdown(response.text[:100] + "... [TRIMMED] ...")

> ## The Colosseum: A Timeless Witness
> 
> Standing tall against the twilight sky, the Colosseum is a tes... [TRIMMED] ...

In [32]:
model = genai.GenerativeModel('gemini-1.5-flash')

messages = [
    {'role':'user',
        'parts': ["Briefly explain how a computer works to a young child."]}
]
response = model.generate_content(messages)

to_markdown(response.text)

> Imagine a computer is like a super smart toy box! Inside, there are lots of tiny parts called "bits" that can be either on or off, like a light switch. 
> 
> When you turn on the computer, it's like opening the toy box. You can then put in different "toys" called programs. These programs tell the computer what to do, like play a game or show you a picture. 
> 
> The computer uses the "bits" to remember all the information from the programs. It's like using the toy box to store all your toys!  
> 
> When you tell the computer to do something, it uses the programs and the bits to work it out, like when you tell a toy robot to walk. 
> 
> So, the computer is like a super smart toy box that can follow instructions and do amazing things! 


In [33]:
messages.append({'role':'model',
                    'parts':[response.text]})

messages.append({'role':'user',
                    'parts':["Okay, how about a more detailed explanation to a high school student?"]})

response = model.generate_content(messages)

to_markdown(response.text)

> A computer is essentially a complex machine that processes information. It operates based on a simple principle: it manipulates binary code, which is a system of 0s and 1s. Here's a breakdown:
> 
> **1. Hardware:** This is the physical part of the computer you can see and touch. It includes:
> 
>    * **CPU (Central Processing Unit):** The brain of the computer. It executes instructions from programs, performs calculations, and manages the flow of data. Think of it as the main conductor of an orchestra.
>    * **RAM (Random Access Memory):** Temporary storage for data and programs currently being used by the computer. It's like a scratchpad for the CPU to work on.
>    * **Storage:** Hard drives, SSDs, etc., store information permanently, like the library of the computer.
>    * **Input/Output Devices:** Keyboard, mouse, monitor, speakers, etc., allow you to interact with the computer and receive information from it.
> 
> **2. Software:** These are the programs and instructions that tell the hardware what to do. They include:
> 
>    * **Operating System (OS):** Manages the hardware resources and provides a user interface (like Windows, macOS, Linux). It acts as the intermediary between hardware and applications.
>    * **Applications:** Programs you use to perform specific tasks, like web browsers, word processors, games, etc.
> 
> **3. The Magic of Binary:** Computers use binary code, a system of 0s and 1s, to represent data and instructions. Each 0 or 1 is a bit, and groups of 8 bits form a byte. This code allows the computer to process information efficiently.
> 
> **4. Processing Information:** When you run a program, the CPU reads instructions from the program and manipulates the data in RAM. It performs calculations, compares values, and makes decisions based on the program's instructions.
> 
> **5. Input, Processing, Output:** This is the fundamental cycle of how a computer works:
> 
>    * **Input:** Data is entered into the computer through input devices.
>    * **Processing:** The CPU manipulates the data based on instructions from the program.
>    * **Output:** The processed information is displayed or sent out through output devices.
> 
> **In Summary:** A computer is a machine that uses binary code to process information based on instructions from software. It interacts with users through input and output devices and relies on various hardware components for its operation.
